# Exploring Meta-T Tetris data from Lindstedt and Gray (2015; 2019)
-------

### Notes on data 

- Download @ https://osf.io/78ebg/
- Three files:
    - "eps" describes the games of 241 participants at the episodic level (behavioural data and scores at each zoid)
    - "gms" provides summary stats (e.g., score, zoid sequence) for each game played in "eps"
    - "quals" includes episodic and summary data for tournament qualifier games played by 44 participants
- All variables described in 2017 paper by Sibert, Lindstedt, and Gray; doi number 10.1111/tops.12225
    - 85 columns in "eps"
    - 15 columns in "gms"
    - 89 columns in "quals"

Import libraries

In [1]:
import os
import sys
import socket
import pingouin as pg
import scipy.stats as scipy

from modules import *
from hmmlearn import hmm

# initialise selected visualisation parameters
sns_styleset()
np.set_printoptions(suppress=True)

/home/ozvar/Dropbox/venvs/hmm_analysis/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/ozvar/Dropbox/venvs/hmm_analysis/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


Import data

In [2]:
df = pd.read_csv(os.path.join('data', 'pca_eps.csv'), header=0, sep=',', low_memory=False)

In [3]:
df.head()

,ts,event_type,SID,ECID,session,game_type,game_number,episode_number,level,score,lines_cleared,curr_zoid,next_zoid,danger_mode,rots,trans,path_length,min_rots,min_trans,min_path,min_rots_diff,min_trans_diff,min_path_diff,u_drops,s_drops,prop_u_drops,initial_lat,drop_lat,avg_lat,tetrises_game,tetrises_level,zoid_rot,zoid_col,zoid_row,all_diffs,all_ht,all_trans,cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cleared,col_trans,column_9,cuml_cleared,cuml_eroded,cuml_wells,d_all_ht,d_max_ht,d_mean_ht,d_pits,deep_wells,eroded_cells,full_cells,jaggedness,landing_height,lumped_pits,matches,max_diffs,max_ht,max_ht_diff,max_well,mean_ht,mean_pit_depth,min_ht,min_ht_diff,move_score,nine_filled,pattern_div,pit_depth,pit_rows,pits,row_trans,tetris,tetris_progress,weighted_cells,wells,criterion,crit_game,study,ep_dur,resp_lat,disarray,well_prep,action_inefficiency,dec_act_latency
0,78.4979,EP_SUMM,101,NIL,2013_6_6_12-23-50,pop_assessment,1,0,0,16,0,O,S,False,0,4,4,0,4,4,0,0,0,17,2,0.8947,724,2083,416.6000,0,0,0,0,2,-2,4,4,0,-2,0,0,0,0,0,0,0,0,2,0,0,0,0,4,2,0.4000,0,0,0,4,2,0,0,4,0,2,1.6000,0,0.4000,0.0000,0,0.4000,0,0,2,0,0,0,2,0,0,6,0,6042,True,TS1,NaN,2083,-5.1342,-1.5865,-0.6677,1.7137
1,88.2783,EP_SUMM,101,NIL,2013_6_6_12-23-50,pop_assessment,1,1,0,24,0,S,O,False,1,7,8,1,3,4,0,4,4,8,10,0.4444,606,4671,505.5455,0,0,1,1,4,-2,9,10,2,-1,-3,0,0,0,0,0,0,0,4,0,0,0,3,5,2,0.5000,1,2,0,8,6,1,1,2,2,4,3.1000,2,0.9000,2.0000,0,0.9000,0,0,4,2,1,1,6,0,0,18,2,6042,True,TS1,9.7805,4671,-3.5601,0.0253,2.0331,2.0280
2,92.9668,EP_SUMM,101,NIL,2013_6_6_12-23-50,pop_assessment,1,2,0,38,0,O,J,False,0,10,10,0,4,4,0,6,6,15,4,0.7895,701,4061,369.1818,0,0,0,8,2,0,13,14,2,-1,-3,0,0,0,0,2,0,0,6,2,0,0,3,4,0,0.4000,0,2,0,12,8,0,1,4,2,4,2.7000,2,1.3000,2.0000,0,1.3000,0,0,5,2,1,1,8,0,0,24,2,6042,True,TS1,4.6885,4061,-3.6738,-0.0468,1.5153,-0.0533
3,99.3719,EP_SUMM,101,NIL,2013_6_6_12-23-50,pop_assessment,1,3,0,51,0,J,S,False,1,4,5,1,1,2,0,3,3,13,6,0.6842,902,2089,621.0000,0,0,1,2,3,0,16,14,2,-1,0,-3,0,0,0,2,0,0,6,2,0,0,3,3,0,0.3000,-1,2,0,16,8,0,0,6,2,4,2.4000,2,1.6000,0.0000,0,1.6000,0,0,6,0,0,0,8,0,0,31,2,6042,True,TS1,6.4051,2089,-3.6946,-0.0797,-0.0159,0.3844
4,106.9975,EP_SUMM,101,NIL,2013_6_6_12-23-50,pop_assessment,1,4,0,58,0,S,O,False,3,3,6,1,3,4,2,0,2,8,8,0.5000,857,6591,820.8889,0,0,1,1,6,0,20,18,4,-1,-2,-3,0,0,0,2,0,0,6,2,0,0,10,4,2,0.4000,0,4,0,20,12,3,0,3,4,6,4.0000,4,2.0000,0.0000,0,2.0000,0,0,6,0,0,0,12,0,0,51,4,6042,True,TS1,7.6256,6591,-2.5802,1.9510,2.4304,2.7268


Add dictionary of DVs for plot legends

In [4]:
components = {
        "well_prep": "Well preparation",
        "action_inefficiency": "Action inefficiency",
        "dec_act_latency": "Decision-action latency",
        "disarray": "Disarray"
    }

## Hidden Markov Models

Set up parameters here
- declare number of states we expect to see
- declare factors to include in the model
- choose player and game to fit model to

In [5]:
# model
n_states = 3
components = {'disarray': 'Disarray', 
                'well_prep': 'Well preparation',
                'action_inefficiency': 'Action inefficiency',
                'dec_act_latency': 'Decision-action latency'}
# data 
player_id = '103'
nth_game = 3

Generate model from above parameters, returning transition matrix and log-likelihood

In [6]:
model, post_prob, X, LL = fit_group_HMM(
                                df=df,
                                n_states=n_states, 
                                components=list(components.keys()),
                                component_labels=components,
                                n_iter=1000,
                                covar_type='diag',
                                null_model=False,
                                nth_fit=1,
                                model_dir='results')

Fitting 3 state model to all games of all players
---------------------------
Transition probabilities:
---------------------------
         State 1  State 2  State 3
State 1   0.9600   0.0300   0.0100
State 2   0.0000   0.9600   0.0300
State 3   0.0400   0.0400   0.9100 

---------------------------
Component means for each state:
---------------------------

                         State 1  State 2  State 3
Disarray                  3.4397  -2.4919   1.0595
Well preparation         -1.7650  -0.3752   2.7918
Action inefficiency      -0.0939  -0.0068   0.1219
Decision-action latency  -0.0700   0.0746  -0.0696 

Switch rate of model is 0.0441

Log-likelihood of model is -2034340.95


Label each episode with the current HMM state

In [7]:
HMM_states_list = model.decode(X)[1].tolist()
df['HMM_state'] = np.array(HMM_states_list)+1

Plot transition matrix

In [8]:
plot_transition_matrix(
    model=model,
    components=components,
    n_states=n_states,
    null_model=False,
    fig_dir='figures/HMMs')

Plot bar chart of fractional occupancy of each state

In [9]:
bar_chart_of_fractional_occupancies(
        df=df,
        n_states=n_states,
        components=components,
        null_model=False,
        cmap='tab10',
        fig_dir='figures/HMMs')

Visualise component scores across each state

In [ ]:
boxplot_state_component_scores(
                df=df,
                components=Z_COMPONENTS,
                n_states=n_states,
                violin=True,
                fig_dir=FIG_DIR)

Visualise time series of state probabilities together with component scores

In [8]:
viz_states(
        df=df, 
        n_states=n_states,
        components=list(components.keys()),
        component_labels=components,
        post_prob=post_prob,
        player_id=player_id, 
        nth_game=nth_game,
        show_plot=True,
        cmap='tab10',
        null_model=False,
        fig_dir='figures/HMMs')

### Visualising multiple HMMs

This block first draws a subsample of tetris games played be the specified players. It then creates containers to store models for each of those games.

In [11]:
# randomly draw from our player sample
players =  ['102']
n_draw = 1
draw = np.random.choice(players, n_draw, replace=False)

# specify number of games for each player
n_games = 5

# specify number of states to iterate over
n_states = [3]

# specify factors and containers
factors = ['well_prep', 'action_inefficiency', 'dec_act_latency']

models, post_probs, LLs = [], [], []
null_models, null_probs, null_LLs = [], [], []
counter = 1

# path to save figures
fig_dir = os.path.join('figures', 'HMMs')

This block iterates over each game, fitting, saving, and visualising HMMs to each game as above, as well as a "null" model consisting of random series for each component

In [12]:
# iterate over subsample
for player in draw:
    for game in range(1, n_games+1):
        for n in n_states:
            print(f'---\nMODEL {counter}: Player {player} Game {game}\n---')
            # fit HMM
            model, post_prob, LL = fit_HMM(df=df,
                                        n_states=n, 
                                        factors=factors, 
                                        factor_labels=factor_labels, 
                                        player_id=player,
                                        nth_game=game,
                                        n_iter=1000,
                                        covar_type='diag')
            models.append(model)
            post_probs.append(post_prob)
            LLs.append(LL)
            # fit HMM after shuffling data in each array - this is our null model for comparison
            print(f'---\nNULL MODEL {counter}: Player {player} Game {game} (shuffled data)\n---')
            null_model, null_prob, null_LL = fit_HMM(df=df,
                            n_states=n, 
                            factors=factors, 
                            factor_labels=factor_labels, 
                            player_id=player,
                            nth_game=game,
                            n_iter=1000,
                            covar_type='diag',
                            null_model=True)
            null_models.append(null_model)
            null_probs.append(null_prob)
            null_LLs.append(null_LL)
            # visualise data
            viz_states(df=df, 
                       n_states=n,
                       factors=factors,
                       factor_labels=factor_labels,
                       post_prob=post_prob,
                       player_id=player, 
                       nth_game=game,
                       fig_dir=fig_dir)
            counter += 1

---
MODEL 1: Player 102 Game 1
---
Fitting 3 state model to game 1 of player 102
---------------------------
Transition probabilities:
---------------------------
         State 1  State 2  State 3
State 1   0.8500   0.1200   0.0300
State 2   0.8000   0.1200   0.0800
State 3   0.0000   0.1200   0.8800 

---------------------------
Component means for each state:
---------------------------

                         State 1  State 2  State 3
Well preparation         -1.1781  -0.7648   1.4065
Action inefficiency      -0.2675   4.2476   0.3668
Decision-action latency  -0.3171  -2.1638  -0.1812 

---------------------------
Fractional occupancy for each state:
---------------------------

State 1 0.6597
State 2 0.1111
State 3 0.2292 

Switch rate of model is 0.2083

Log-likelihood of model is -727.73


ValueError: too many values to unpack (expected 3)

## Match states across models

- States can switch labels every time a model is fitted to new data - i.e. a state 1 determined principally by action inefficiency might be labelled as state 3 in a different game.
- We attempt to match states here by correlating the vector of component means for each state between models (2 models at a time).
- Two states are "matched" if their correlation is above a specified threshold.
- If each state in model A has one unique match in model B, the matching is successful and we can claim the existence of some unique states co-existing in models A and B.
- If the unique matching is successful between every pair of models for a given player, we can further claim the existence of some unique internal states associated with performance for this player's tetris experience.

In [ ]:
n_states = 3
a = models[1].means_
b = models[2].means_

corr_mat = np.zeros((n_states, n_states))
for i in range(n_states):
    for j in range(n_states):
        corr = scipy.pearsonr(a[i], b[j])[0]
        corr_mat[i, j] = corr

In [ ]:
corr_mat

In [ ]:
n = 3
threshold = 0.8

total = 0
successes = 0 
for index_a, model_a in enumerate(models):
    for index_b, model_b in enumerate(models[index_a+1:]):
        a = model_a.means_
        b = model_b.means_
        corr_mat = cross_corr(n, a, b)
        state_names = [f'State {i+1}' for i in range(n_states)]
        corr_df = pd.DataFrame(corr_mat)
        corr_df.index = corr_df.columns = state_names
        print('---')
        print(f'Correlating state vectors of models {index_a+1} and {index_a+index_b+2}')
        total += 1
        if check_unique_state_matches(n, corr_df, threshold):
            successes += 1

print(f'\nTotal successful matches = {successes} / {total}')

Work-in-progress blocks identifying and visualising unique matches between any two given models

In [ ]:
game_a = 0
game_b = 1

a = models[game_a].means_
b = models[game_b].means_

corr_mat = cross_corr(n, a, b)
state_names = [f'State {i+1}' for i in range(n)]
corr_df = pd.DataFrame(corr_mat)
corr_df.index = corr_df.columns = state_names

In [ ]:
print('Model A')
print(tabulate_means(models[game_a], factors, factor_labels, n), '\n')
print('---')
print('Model B')
print(tabulate_means(models[game_b], factors, factor_labels, n), '\n')

In [ ]:
check_unique_state_matches(n, corr_df, threshold)

In [ ]:
corr_df

In [ ]:
ax = sns.heatmap(corr_df, center=0)
ax.set_ylabel('Model A')
ax.set_xlabel('Model B')

plt.show()

### Visualise state similarities in 2D space using MDS

In [ ]:
from sklearn import manifold
from sklearn.metrics import euclidean_distances

Fit HMM to first game of each player and concatenate all vectors of means describing each state

In [ ]:
nth_game = 1
players = df['SID'].unique()
states = []
for player_id in players:
    model, post_prob, LL = fit_HMM(df=df,
                                n_states=3, 
                                factors=factors, 
                                factor_labels=factor_labels, 
                                player_id=player,
                                nth_game=game,
                                verbose=False,
                                n_iter=1000,
                                covar_type='diag')
    states.append(model.means_)

states = np.concatenate(states)

Initialise MDS and feed states to visualise similarities

In [ ]:
seed = np.random.RandomState(seed=3)

# sklearn does metric MDS by default, but non-metric MDS is possible using the "metric" boolean
mds = manifold.MDS(
    n_components=2,
    metric=True,
    max_iter=3000,
    eps=1e-9,
    random_state=seed,
    dissimilarity="euclidean",
    n_jobs=1,
)

pos2d = mds.fit(states).embedding_
pos2d.shape # check correct data structure is outputted

In [ ]:
plt.scatter(pos2d[:,0], pos2d[:,1])
plt.xlabel('Coordinate 1')
plt.ylabel('Coordinate 2')
plt.show()

Are these four distinct clusters of similar sizes, or are there outliers?
We plot histograms of x and y coordinates to investigate.

In [ ]:
plt.hist(pos2d[:,0])
plt.show()

In [ ]:
plt.hist(pos2d[:,1])
plt.show()

Try the same in 3D space

In [ ]:
seed = np.random.RandomState(seed=3)

mds = manifold.MDS(
    n_components=3,
    max_iter=3000,
    eps=1e-9,
    random_state=seed,
    dissimilarity="euclidean",
    n_jobs=1,
)

pos3d = mds.fit(states).embedding_
pos3d.shape # check correct data structure is outputted

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D(pos3d[:,0], pos3d[:,1], pos3d[:,2], cmap='Greys')
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Dimension 3')

### Repeat MDS with bootstrapped data

In [ ]:
nth_game = 1
players = df['SID'].unique()
states = []
for player_id in players:
    model, post_prob, LL = fit_HMM(df=df,
                                n_states=3, 
                                factors=factors, 
                                factor_labels=factor_labels, 
                                player_id=player,
                                nth_game=game,
                                verbose=False,
                                n_iter=1000,
                                covar_type='diag',
                                null_model=True)
    states.append(model.means_)

states = np.concatenate(states)

Initialise MDS and feed states to visualise similarities

In [ ]:
seed = np.random.RandomState(seed=3)

# sklearn does metric MDS by default, but non-metric MDS is possible using the "metric" boolean
mds = manifold.MDS(
    n_components=2,
    metric=True,
    max_iter=3000,
    eps=1e-9,
    random_state=seed,
    dissimilarity="euclidean",
    n_jobs=1,
)

pos2d = mds.fit(states).embedding_
pos2d.shape # check correct data structure is outputted

Histograms of x and y coordinates

In [ ]:
plt.hist(pos2d[:,0])
plt.show()

In [ ]:
plt.hist(pos2d[:,1])
plt.show()

In [ ]:
plt.scatter(pos2d[:,0], pos2d[:,1])
plt.xlabel('Coordinate 1')
plt.ylabel('Coordinate 2')
plt.show()